In [90]:
import pandas as pd
import numpy as np 
import re

In [91]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Main V4.")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Slot Description V4.")

In [92]:
# print slot intent dataframe
df.head()

,QUESTION_LABEL,TARGET_QUESTION,INTENT_DESCRIPTION,EXAMPLE_SENTENCE,QUESTION_SHOULD_CONTAIN
0,buy,Where can I buy a @parts for my @appliance?\nI...,The sentence specifies that the user wants to ...,Where can I buy a glass tray for my microwave?...,@parts @appliance
1,locate,Where can I locate the @object on the @applian...,The sentence specifies that the user wants to ...,Where can I locate the model number on the mic...,@object @appliance
2,Purpose,What is the purpose of the @with_feature_objec...,The sentence asks for a purpose of a feature.\...,What is the purpose of the warming center feat...,@with_feature_object @appliance
3,cook,What must I do to cook @food with the @applian...,The sentence asks how to prepare a certain typ...,What must I do to cook popcorn with the microw...,@food @appliance
4,Information Code,What does @info_code mean on my @appliance?\nM...,The sentence is asking about the\nmeaning of t...,What is the c-d0 mean on my oven?\nMy microwav...,@info_code @appliance


In [93]:
# print slot dataframe
slot_df.head()

,Slot,Description,Examples,Extra Explanation
0,@parts,A Physical part from an appliance.\nSomething ...,"door, light, storage drawer, exhaust adaptor, ...",The following slot values are items that can b...
1,@appliance,Any type of appliance. This slot is \nused to ...,microwavawe,This slot describes the appliance that the use...
2,@object,Anything that is physical.,"door, light, storage-drawer, exhaust-adaptor, ...",This slot is the larger set which contains all...
3,@feature,Features that has settings that can be \nmanip...,"cooking, boiling, melting, surface-cooking, so...",NaN
4,@with_feature,Features that has settings that can be \nmanip...,"cooking feature, boiling feature, melting feat...",NaN


In [94]:
# Slot dataframe trimmed so that the Description is removed. 
slot_df_ = slot_df.iloc[:,[0,2]]

In [95]:
# replace the Example column with an array of the values separated by a comma
slot_df_['Examples'] = slot_df_["Examples"].apply(lambda x: x.split(', '))

C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [96]:
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_.Slot, slot_df_.Examples))

In [97]:
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))

In [98]:
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]
# for s in range(len(temp)):
#     for j in range(len(temp[s])):
#         print(temp[s][j])
#     print("-----------------------------------------------")

In [99]:
df

,QUESTION_LABEL,TARGET_QUESTION,INTENT_DESCRIPTION,EXAMPLE_SENTENCE,QUESTION_SHOULD_CONTAIN
0,buy,"[Where can I buy a @parts for my @appliance?, ...",The sentence specifies that the user wants to ...,Where can I buy a glass tray for my microwave?...,@parts @appliance
1,locate,[Where can I locate the @object on the @applia...,The sentence specifies that the user wants to ...,Where can I locate the model number on the mic...,@object @appliance
2,Purpose,[What is the purpose of the @with_feature_obje...,The sentence asks for a purpose of a feature.\...,What is the purpose of the warming center feat...,@with_feature_object @appliance
3,cook,[What must I do to cook @food with the @applia...,The sentence asks how to prepare a certain typ...,What must I do to cook popcorn with the microw...,@food @appliance
4,Information Code,"[What does @info_code mean on my @appliance?, ...",The sentence is asking about the\nmeaning of t...,What is the c-d0 mean on my oven?\nMy microwav...,@info_code @appliance
5,Options,[What are the different options of the @with_f...,The sentence asks about the different settings...,What are the different options of the Cook fea...,@with_feature @appliance
6,image,[What does the @object look like in the @appli...,The sentence asks how an object looks like.,What does the Wire racks Look Like in the Micr...,@object @appliance
7,remove,[How do I remove @removeable_parts from the @a...,The sentence asks how to remove something\nfro...,How do I remove burned-on residue from the ove...,@removeable_parts @appliance
8,reinstall,[How do I reinstall @replaceable_parts from th...,The sentence asks how to reinstall something\n...,How do I reinstall the storage drawer from the...,@replaceable_parts @appliance
9,appropriate,"[Is it safe to use @cookware in a @appliance?,...",A sentence that asks if it is safe to use a ce...,Is it safe to use Glassware in the microwave?\...,@cookware @appliance


In [100]:
#creating the slots for each dictionary
slots = df.iloc[:,4].apply(lambda x: x.split(' '))

In [101]:
target_dic = df.iloc[:,[0,1]]

In [102]:
# for s in range(len(target)):
#     for j in range(len(target[s])):
#         print(target[s][j])
#     print("-----------------------------------------------")

In [103]:
# Print out all the sentences. 
# for s in range(len(temp)): #Intent number
#     for j in range(len(temp[s])): #Sentence in the intent
#         for i in range(len(slot_dict[slots[s][0]])): #Parts slot 1
#             for k in range(len(slot_dict[slots[s][1]])):
#                 x = temp[s][j]
#                 x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
#                 x = re.sub(slots[s][1], slot_dict[slots[s][1]][k], x)
#                 print(x)        
#     print("-----------------------------------------------")

In [104]:
i = 0
for s in range(len(target)): #Intent number
    print(len(slots[s]))
    for j in range(len(target[s])): #Sentence in the intent
        if (len(slots[s]) == 2):
            print("two slots")
            i += two_slots(s,j)    
        elif (len(slots[s]) == 3):
            print("three slots")
            i += three_slots(s,j)
        elif (len(slots[s]) == 1):
            print("one slot")
            i += one_slot(s,j)
    print("-----------------------------------------------")
    
print(i)

2
two slots
Where can I buy a door for my microwavawe?
Where can I buy a light for my microwavawe?
Where can I buy a storage drawer for my microwavawe?
Where can I buy a exhaust adaptor for my microwavawe?
Where can I buy a grease filter for my microwavawe?
Where can I buy a characol filter for my microwavawe?
Where can I buy a wire rack for my microwavawe?
Where can I buy a air fry tray for my microwavawe?
Where can I buy a glass tray for my microwavawe?
Where can I buy a grill rack for my microwavawe?
Where can I buy a screw for my microwavawe?
Where can I buy a bracket for my microwavawe?
two slots
I want to get a new door for my microwavawe.
I want to get a new light for my microwavawe.
I want to get a new storage drawer for my microwavawe.
I want to get a new exhaust adaptor for my microwavawe.
I want to get a new grease filter for my microwavawe.
I want to get a new characol filter for my microwavawe.
I want to get a new wire rack for my microwavawe.
I want to get a new air fry t

In [105]:
def two_slots(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
            x = re.sub(slots[s][1], slot_dict[slots[s][1]][k], x)
            p += 1
            print(x)    
    return p

In [106]:
def three_slots(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            for l in range(len(slot_dict[slots[s][2]])):#Slot 3
                x = target[s][j]
                x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
                x = re.sub(slots[s][1], slot_dict[slots[s][1]][k], x)
                x = re.sub(slots[s][2], slot_dict[slots[s][2]][l], x)
                p += 1
                print(x)   
    return p

In [107]:
def one_slot(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
            p += 1
            print(x)    
    return p

In [108]:
intent_dict = df.QUESTION_LABEL.to_dict()

In [189]:
def print_all(intent_number = len(target), single = False):
    i = 0
    start = 0
    end = intent_number
    if (single == True):
        start = intent_number - 1
        end = intent_number
    for s in range(start,end): #Intent number
        print("-----------------------------------------------")
        print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
        for j in range(len(target[s])): #Sentence in the intent
            if (len(slots[s]) == 2):
                #print("two slots")
                i += two_slots(s,j)    
            elif (len(slots[s]) == 3):
                #print("three slots")
                i += three_slots(s,j)
            elif (len(slots[s]) == 1):
                #print("one slot")
                i += one_slot(s,j)
            print("-----------------------------------------------")
    print("-----------------------------------------------")

    print(i)

In [192]:
# print_all(single = False) # Print everything
print_all(15, single=True)
#print_all(single=False)

-----------------------------------------------
Intent: Speed Level | number of slots: 2
How do I set the speed levels of the fan on my microwavawe?
How do I set the speed levels of the vent on my microwavawe?
-----------------------------------------------
-----------------------------------------------
2


In [111]:
def print_all(intent_number = len(target), single = False):
    i = 0
    start = 0
    end = intent_number
    if (single == True):
        start = intent_number - 1
        end = intent_number
    for s in range(start,end): #Intent number
        print("-----------------------------------------------")
        print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
        for j in range(len(target[s])): #Sentence in the intent
            if (len(slots[s]) == 2):
                #print("two slots")
                i += two_slots(s,j)    
            elif (len(slots[s]) == 3):
                #print("three slots")
                i += three_slots(s,j)
            elif (len(slots[s]) == 1):
                #print("one slot")
                i += one_slot(s,j)
            print("-----------------------------------------------")
    print("-----------------------------------------------")

    print(i)

In [112]:
import os
if not os.path.exists('data_1'):
    open('data_1', 'w').close() 

In [113]:
myFile = open('data_1','w')

In [114]:
def boi_one_slot(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
            p += 1
            print(x)  
    return x, p

In [115]:
def boi_three_slots(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            for l in range(len(slot_dict[slots[s][2]])):#Slot 3
                x_original = target[s][j]
                x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x_original)
                x = re.sub(slots[s][1], slot_dict[slots[s][1]][k], x)
                x = re.sub(slots[s][2], slot_dict[slots[s][2]][l], x)
                x_boi = ""
                p += 1
                print(x)   
    return x, p

In [116]:
def boi(intent_number = len(target), single = False):
    line = ""
    start = 0
    end = intent_number
    if (single == True):
        start = intent_number - 1
        end = intent_number
    for s in range(start,end): #Intent number
        print("-----------------------------------------------")
        print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
        for j in range(len(target[s])): #Sentence in the intent
            if (len(slots[s]) == 2):
                line,_ = boi_two_slots(s,j)    
            elif (len(slots[s]) == 3):
                line,_ = boi_three_slots(s,j)
            elif (len(slots[s]) == 1):
                line,_ = boi_one_slot(s,j)
            print("-----------------------------------------------")
    print("-----------------------------------------------")

In [117]:
import copy
slot_dict_boi = copy.deepcopy(slot_dict)

In [118]:
slot_dict_boi

{'@parts': ['door',
  'light',
  'storage drawer',
  'exhaust adaptor',
  'grease filter',
  'characol filter',
  'wire rack',
  'air fry tray',
  'glass tray',
  'grill rack',
  'screw',
  'bracket'],
 '@appliance': ['microwavawe'],
 '@object': ['door',
  'light',
  'storage-drawer',
  'exhaust-adaptor',
  'grease-filter',
  'characol-filter',
  'wire-rack',
  'air-fry tray',
  'glass-tray',
  'grill-rack',
  'cooking-button',
  'melting-button',
  'surface-cooking-button',
  'softening-button',
  'airfry-button',
  'boiling-button',
  'keep-warm-button',
  'dehydrate-button',
  'autocook-button',
  'grill-button',
  'microwave-grill-button',
  'defrost-button',
  'delay-start-button',
  'fan-button',
  'fan',
  'power-defrost-button',
  'reheat-button',
  'auto-defrost-button',
  'self-clean-button',
  'timer-button',
  'smart-control-button',
  'lamp',
  'lamp-button',
  'lightings',
  'clock-settings',
  'child-lock',
  'timer',
  'echo-mode-button',
  'child-lock-button',
  'beepe

In [119]:
for key,value in slot_dict.items():
    print(key)
    list_slot = []
    for i in value:
        s = i.split()
        for j in range(len(s)):
            print(s[j])
            if j == 0:
                s[j] = s[j] + ":B-" + key[1:]
            else:
                s[j] = s[j] + ":I-" + key[1:]
        i = ' '.join(s)
        list_slot.append(i)
    slot_dict_boi[key] = list_slot
                
print(slot_dict_boi)

@parts
door
light
storage
drawer
exhaust
adaptor
grease
filter
characol
filter
wire
rack
air
fry
tray
glass
tray
grill
rack
screw
bracket
@appliance
microwavawe
@object
door
light
storage-drawer
exhaust-adaptor
grease-filter
characol-filter
wire-rack
air-fry
tray
glass-tray
grill-rack
cooking-button
melting-button
surface-cooking-button
softening-button
airfry-button
boiling-button
keep-warm-button
dehydrate-button
autocook-button
grill-button
microwave-grill-button
defrost-button
delay-start-button
fan-button
fan
power-defrost-button
reheat-button
auto-defrost-button
self-clean-button
timer-button
smart-control-button
lamp
lamp-button
lightings
clock-settings
child-lock
timer
echo-mode-button
child-lock-button
beeper
smart-controls
left-rear
right-rear
left-front
right-front
heating-center
surface-controls
warming-center
vent-button
model-number
vent
sorage-drawer
broil-oven-heater
convection-system
option-key
touch-pad
number-pad
@feature
cooking
boiling
melting
surface-cooking
softe

In [120]:
boi(1,single=True)

-----------------------------------------------
Intent: buy | number of slots: 2
Where:o can:o I:o buy:o a:o door:B-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o light:B-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o storage:B-parts drawer:I-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o exhaust:B-parts adaptor:I-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o grease:B-parts filter:I-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o characol:B-parts filter:I-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o wire:B-parts rack:I-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o air:B-parts fry:I-parts tray:I-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o glass:B-parts tray:I-parts for:o my:o microwavawe:B-appliance?
Where:o can:o I:o buy:o a:o grill:B-parts rack:I-parts for:o my:o microwavawe:B-appliance?
Where:o ca

In [121]:

def boi_two_slots(s,j):
    p = 0
    x_boi = ""
    for i in range(len(slot_dict_boi[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict_boi[slots[s][1]])):#Slot 2
            x_original = target[s][j]
            x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
            x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
#             x_boi = re.sub(slots[s][0], slot_dict[slots[s][0]][i] + ":B-"+slots[s][0][1:], x_original)
#             x_boi = re.sub(slots[s][1], slot_dict[slots[s][1]][k] + ":B-"+slots[s][1][1:], x_boi)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:o', x) 
            #print(re.findall(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:o', x_boi))
            #print(re.findall(r'\w+',x_boi))
            #print(slots[s][0], slot_dict[slots[s][0]][i])
            p += 1
            print(x_boi)
           # print(x_boi)
           # print(x_boi_2)
    return x, p

In [122]:
slot_dict_boi

{'@parts': ['door:B-parts',
  'light:B-parts',
  'storage:B-parts drawer:I-parts',
  'exhaust:B-parts adaptor:I-parts',
  'grease:B-parts filter:I-parts',
  'characol:B-parts filter:I-parts',
  'wire:B-parts rack:I-parts',
  'air:B-parts fry:I-parts tray:I-parts',
  'glass:B-parts tray:I-parts',
  'grill:B-parts rack:I-parts',
  'screw:B-parts',
  'bracket:B-parts'],
 '@appliance': ['microwavawe:B-appliance'],
 '@object': ['door:B-object',
  'light:B-object',
  'storage-drawer:B-object',
  'exhaust-adaptor:B-object',
  'grease-filter:B-object',
  'characol-filter:B-object',
  'wire-rack:B-object',
  'air-fry:B-object tray:I-object',
  'glass-tray:B-object',
  'grill-rack:B-object',
  'cooking-button:B-object',
  'melting-button:B-object',
  'surface-cooking-button:B-object',
  'softening-button:B-object',
  'airfry-button:B-object',
  'boiling-button:B-object',
  'keep-warm-button:B-object',
  'dehydrate-button:B-object',
  'autocook-button:B-object',
  'grill-button:B-object',
  'micr

In [154]:
slot_array = [] 
for key,value in slot_dict_boi.items():
    for i in value:
        slot_array = slot_array + i.split()
slot_array.sort()
slot_vocab = []
for i in slot_array:
    slot_vocab.append(i.split(':',1)[1])
    
slot_vocab.sort()
slot_vocab = list(dict.fromkeys(slot_vocab))

for i in slot_vocab:
    print(i)

B-appliance
B-cleanable
B-cooking_feature
B-cookware
B-feature
B-feature_cancel
B-feature_object
B-feature_on_off
B-feature_power_level
B-feature_speed_level
B-food
B-info_code
B-object
B-parts
B-removeable_parts
B-replaceable_parts
B-with_feature
B-with_feature_object
I-cleanable
I-cooking_feature
I-cookware
I-feature_on_off
I-food
I-info_code
I-object
I-parts
I-removeable_parts
I-replaceable_parts
I-with_feature
I-with_feature_object


In [ ]:
f = open('temp_vocab.slot','w')
for value in slot_vocab:
    f.write(value + "\n")
f.write("O")
f.close()

In [165]:
import random

In [185]:
my_file = open('file3','r')
content = my_file.readlines()
random.shuffle(content)
print(len(content))
train = content[:int(len(content)*0.7)]
test = content[int(len(content)*0.7):int(len(content)*0.9)]
valid = content[int(len(content)*0.9):]
print(len(train))
print(len(test))
print(len(valid))

f = open('temp_train','w')
for i in train:
    f.write(i)
    
f = open('temp_test','w')
for i in test:
    f.write(i)
    
f = open('temp_valid','w')
for i in test:
    f.write(i)

705
493
141
71


In [168]:
import random, sys
x = open(sys.argv[1]).readlines()
random.shuffle(x)
print ''.join(x)," shuffle_mylines.txt  

SyntaxError: invalid syntax (<ipython-input-168-acd8a7b399d2>, line 4)

In [123]:
intent_dict

{0: 'buy',
 1: 'locate',
 2: 'Purpose',
 3: 'cook',
 4: 'Information Code',
 5: 'Options',
 6: 'image',
 7: 'remove',
 8: 'reinstall',
 9: 'appropriate',
 10: 'clean',
 11: 'how_to',
 12: 'set',
 13: 'Time',
 14: 'Speed Level',
 15: 'Power Level',
 16: 'on',
 17: 'off'}

In [124]:
f = open('temp_vocab.intent','w')
for key,value in intent_dict.items():
    f.write(value + "\n")
f.close()

In [125]:
def boi_one_slot_write(s,j,f):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict[slots_boi[s][0]][i], x)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            p += 1
            f.write(x_boi[:-1] + " <=> " + intent_dict[s] + "\n")
            print(x)  
    return x, p

def boi_two_slots_write(s,j,f):
    p = 0
    x_boi = ""
    for i in range(len(slot_dict_boi[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict_boi[slots[s][1]])):#Slot 2
            x_original = target[s][j]
            x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
            x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
#             x_boi = re.sub(slots[s][0], slot_dict[slots[s][0]][i] + ":B-"+slots[s][0][1:], x_original)
#             x_boi = re.sub(slots[s][1], slot_dict[slots[s][1]][k] + ":B-"+slots[s][1][1:], x_boi)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            #print(re.findall(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:o', x_boi))
            #print(re.findall(r'\w+',x_boi))
            #print(slots[s][0], slot_dict[slots[s][0]][i])
            p += 1
            f.write(x_boi[:-1]+" <=> " + intent_dict[s] + "\n")
            print(x_boi)
           # print(x_boi)
           # print(x_boi_2)
    return x, p

def boi_three_slots_write(s,j,f):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            for l in range(len(slot_dict[slots[s][2]])):#Slot 3
                x_original = target[s][j]
                x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
                x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
                x = re.sub(slots[s][2], slot_dict_boi[slots[s][2]][l], x)
                x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
                p += 1
                f.write(x_boi[:-1]+" <=> "+ intent_dict[s] + "\n")
                print(x)   
    return x, p

In [126]:
def boi_save(intent_number = len(target), single = False):
    f = open("file3", "w")
    line = ""
    start = 0
    end = intent_number
    if (single == True):
        start = intent_number - 1
        end = intent_number
    for s in range(start,end): #Intent number
        print("-----------------------------------------------")
        print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
        for j in range(len(target[s])): #Sentence in the intent
            if (len(slots[s]) == 2):
                line,_ = boi_two_slots_write(s,j,f)    
            elif (len(slots[s]) == 3):
                line,_ = boi_three_slots_write(s,j,f)
            elif (len(slots[s]) == 1):
                line,_ = boi_one_slot_write(s,j,f)
            print("-----------------------------------------------")
    print("-----------------------------------------------")

In [127]:
boi_save()

-----------------------------------------------
Intent: buy | number of slots: 2
Where:O can:O I:O buy:O a:O door:B-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O light:B-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O storage:B-parts drawer:I-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O exhaust:B-parts adaptor:I-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O grease:B-parts filter:I-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O characol:B-parts filter:I-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O wire:B-parts rack:I-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O air:B-parts fry:I-parts tray:I-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O glass:B-parts tray:I-parts for:O my:O microwavawe:B-appliance?
Where:O can:O I:O buy:O a:O grill:B-parts rack:I-parts for:O my:O microwavawe:B-appliance?
Where:O ca

Intent: Purpose | number of slots: 2
What:O is:O the:O purpose:O of:O the:O cooking:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O is:O the:O purpose:O of:O the:O boiling:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O is:O the:O purpose:O of:O the:O melting:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O is:O the:O purpose:O of:O the:O surface-cooking:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O is:O the:O purpose:O of:O the:O softening:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O is:O the:O purpose:O of:O the:O airfry:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O is:O the:O purpose:O of:O the:O boil:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O is:O 

What:O are:O the:O instructions:O in:O using:O the:O auto-defrost:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O are:O the:O instructions:O in:O using:O the:O baking-feature:B-with_feature_object on:O the:O microwavawe:B-appliance?
What:O are:O the:O instructions:O in:O using:O the:O self-clean:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O are:O the:O instructions:O in:O using:O the:O timer:B-with_feature_object on:O the:O microwavawe:B-appliance?
What:O are:O the:O instructions:O in:O using:O the:O smart-control:B-with_feature_object feature:I-with_feature_object on:O the:O microwavawe:B-appliance?
What:O are:O the:O instructions:O in:O using:O the:O lamp:B-with_feature_object on:O the:O microwavawe:B-appliance?
What:O are:O the:O instructions:O in:O using:O the:O lightings:B-with_feature_object on:O the:O microwavawe:B-appliance?
What:O are:O the:O instructions:O in:O using:O the:O clock-

NameError: name 'slots_boi' is not defined

In [ ]:
Regex.Replace(input, @"[abc]", m => string.Format(@"\{0}", m.Value))

In [ ]:
import re
text = 'Do you get it yet?'
re.sub(r'(\w+)', r'\1:o', text)
